## Introduction

Tests LangChain with multiple tools and different models (e.g. OpenAI - paid; Vicuna/Falcon - local, free).

Selected outputs recorded here: https://amandakoh.notion.site/testing-7e0672230e9942f39618824120bf8aa2?pvs=4

In [ ]:
import tqdm

from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, AutoTokenizer, AutoConfig
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer
from langchain.agents import load_tools
import torch
import os

# Load models

## Vicuna (local, free)

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
# import model
llama_tokenizer = LlamaTokenizer.from_pretrained("LLMModels/vicuna/13B")
llama_model = LlamaForCausalLM.from_pretrained("LLMModels/vicuna/13B", device_map="auto")    

# use model as a text-generation model in a pipeline
pipe = pipeline(
        "text-generation",
        model=llama_model, 
        tokenizer=llama_tokenizer, 
        max_length=2000,
        temperature=0.0,
        top_p=0.95,
        repetition_penalty=1.2
    )

In [ ]:
# create LLM
llm = HuggingFacePipeline(pipeline=pipe)

## Falcon (local, free)

In [ ]:
model = "LLMModels/falcon/falcon-40b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    max_new_tokens=100,
)

In [ ]:
# create LLM
llm = HuggingFacePipeline(pipeline = pipeline, 
                          model_kwargs = {'temperature':0.5})

## OpenAI (API, paid)

In [ ]:
# os.environ["OPENAI_API_KEY"] = ADD API KEY!!!

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo', max_retries=1)

# Random questions

In [ ]:
questions = [
#             "what is a group of crows called?",
#              "Alice has a parrot. What animal is Alice's pet?",
             "what is the meaning of life?",

#             "Describe a cat.",
            #  "The expected response for a highly intelligent chatbot to 'Describe a cat.' is '"

#             "what is the capital of france?",
            # "The answer to the question \"What is the capital of France?\" is \"",
            # "User: What is the capital of France? \n Bot: ",
            # "User: Who was the president of US in 1960? \n Bot: "
]

for query in questions:
    print("Query:", query)
    print("Ans:",llm(query))
    print()

# Tools

## Multiplier tool

In [ ]:
def multiplier(string):
    a, b = string.split(",")
    return int(a) * int(b)

In [ ]:
multiplierPrompt = """Example:

[begin]

Question: What is 5 times 8?
Thought: I must use the Multiplier tool to calculate 5 times 8
Action: Multiplier
Action Input: 5,8
Observation: 40

Thought: I now know the answer
Final Answer: 40

[end]

"""

## Defense info tool

In [ ]:
def defenseInformation(string):
    string = string.lower()
    if string == "army":
        return """"The Singapore Army is the land service branch of the Singapore Armed Forces (SAF). The largest of the four branches of the SAF, the Singapore Army traces its origins to the 1st Battalion, Singapore Infantry Regiment (1 SIR), which was formed in 1957, when Singapore was still under British colonial rule. After Singapore's independence on 9 August 1965, the Singapore Army Bill was passed in Parliament on 23 December 1965, and National Service (NS) was subsequently introduced in 1967. Mostly made up of conscripts, the Singapore Army can mobilise all operationally-ready military reservists in the event of war or national exigencies."""
    if string == "navy":
        return "The Republic of Singapore Navy (RSN) is the naval service branch of the Singapore Armed Forces (SAF) responsible for defending the country against any sea-borne threats, and the protection of its sea lines of communications, that would compromise Singapore as a global trading hub. The RSN traces its origins to the Royal Navy when Singapore was still a Crown colony of the British Empire. After Singapore's independence from Malaysia in 1965, the service was formally established in 1967, and had undergone a substantial modernisation ever since—which led them into becoming the most powerful navy in Southeast Asia."
    if string == "air forces":
        return "The Republic of Singapore Air Force (RSAF) is the aerial service branch of the Singapore Armed Forces (SAF) responsible for controlling and defending the airspace of the country, and providing air support to the Army and Navy. It was established in 1968 as the Singapore Air Defence Command (SADC) before renaming to its current name in 1975.[5] As the most powerful air force in Southeast Asia, the RSAF has undertaken a significant role in Singapore's military defence strategy since its formation."
    if string == "digital intelligence":
        return "The Digital and Intelligence Service (DIS) is the digital service branch of the Singapore Armed Forces (SAF) responsible for providing military intelligence to the armed forces, building up the country's digital defence capabilities, and protecting the psychological defence of its military personnel. It was established on 28 October 2022, in response to the increased number of attacks by non-state actors, and the resulting damage from the Russian-Ukrainian cyberwarfare."
    return "Invalid input"

In [ ]:
defenseInformationPrompt = """Example:

[begin]

Question: How many people does Singapore's army have?
Thought: I must use the Defense Information tool to learn more about the army
Action: Defense Information
Action Input: army
Observation: The Singapore Army is the land service branch of the Singapore Armed Forces, and has 45,000 troops.

Thought: I now know the answer.
Fianl Answer: 45,000

[end]

"""

## News tool

In [ ]:
def latestNews(topic):
    # in reality this would actually search up a news article, but rn it's just a random fixed article
    news = "The Philippines is, for the first time, allowing tourism in areas it controls in the hotly contested Spratly Islands of the disputed South China Sea. The islands which the Philippines has laid claim on are in the northeastern part of the Spratly archipelago, an area locally known as the Kalayaan Island Group of the West Philippine Sea. The Spratly Islands consist of more than 100 small islands or reefs surrounded by rich fishing grounds and potentially by gas and oil deposits. They are claimed in their entirety by China, Taiwan, and Vietnam, while portions are claimed by Malaysia and the Philippines. The seven-day tour aboard a dive yacht is dubbed the Great Kalayaan or Freedom Expedition, and costs more than US$2,000 per person."
    end = "\nPlease summarise this news for the user as a final answer."

    return "The latest news on " + topic + " is " + news + end
    

## Make an incident report

In [ ]:
def makeIncidentReport(report):
    # code that actually makes an incident report
    return "Incident report made successfully"

## Retrieve latest incident report

In [ ]:
def retrieveIncidentReport(string):
    return "The last incident report was 'Two unauthorised men spotted in camp'"

## Find document

In [ ]:
def searchForDocument(string):
    return "Document contents of {string}: The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English- to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data."

# Running

In [ ]:
tools = [
    Tool(
        name = "Latest News",
        func = latestNews,
        description="useful for when you need to get the latest news on any specified topic. The input to this tool should be the topic or category of news that should be retrieved. For example, a topic could be the Russia-Ukraine war.",
    ),
    Tool(
        name = "Defense Information",
        func=defenseInformation,
        description="useful for when you need to find background information about Singapore's defense force, including the army, navy, air force and the digital and intelligence service. The input to this tool should be one of [army, navy, air force, digital intelligence] depending on which branch you are interested in."
    ),
    Tool(
        name = "Multiplier",
        func=multiplier,
        description="useful for when you need to multiply two numbers together. The input to this tool should be a comma separated list of numbers of length two, representing the two numbers you want to multiply together. For example, 1,2 would be the input if you wanted to multiply 1 by 2."
    ),
    Tool(
        name = "Make Incident Report",
        func=makeIncidentReport,
        description="useful for when the user asks for an incident report to be made. The input to this tool is the string that the user inputs."
    ),
    Tool(
        name = "Retrieve Incident Report",
        func=retrieveIncidentReport,
        description="only use when the specifically requests for the latest incident report. This tool does not require an input."
    ),
    Tool(
        name = "Search for Document",
        func=searchForDocument,
        description="useful for searching for a specific document such as a PDF or text file. The input to this tool is the search query."
    ),
]

## Initialize agent (can modify prompt)

In [ ]:
vicunaSuffix = """Remember, if you don't need a tool to answer the question, end with 'Final Answer: '. Always start with a thought before taking any action. Now begin!

Question: {input}
Thought: {agent_scratchpad}
"""
# vicunaPrompt = "Here are some examples:\n\n" + multiplierPrompt + defenseInformationPrompt + vicunaSuffix

suffix = """Only use a tool if it is relevant to the question and only the tools available. If you don't need a tool, write 'Final Answer: '. 'None' is not a valid tool. Now begin!

Question: {input}
Thought: {agent_scratchpad}
"""

# For vicuna agent might need to remind it to use action input (because it tends to write input action instead)

agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         handle_parsing_errors=True,
                         agent_kwargs = {
                             'suffix': suffix
                         })

In [ ]:
# Print out template of the agent to check
print(agent.agent.llm_chain.prompt.template)

## Ask question

In [ ]:
question = "find the latest news in the south china sea, and make an incident report about that"
print(agent.run(question))